In [9]:
import tensorflow as tf
import numpy as np
import pandas as pd
import os
import pathlib
from google.colab import drive

# 挂载 Google Drive
drive.mount('/content/drive')

# 设置文件路径
counting_data_dir = '/content/drive/MyDrive/Machine Vision/Minne Apple Count/counting_data/counting/'
image_folder_counting_trains = counting_data_dir + 'train/images/'
image_folder_counting_test = counting_data_dir + '/test/images/'
image_folder_counting_val = counting_data_dir + 'val/images/'

# 数据集参数
batch_size = 64
shuffle_size = 1000
img_height = 180
img_width = 180

# 读取标签配对文件
# train
train_ground_truth = pd.read_csv(counting_data_dir + 'train/train_ground_truth.txt')
train_image_names = train_ground_truth['Image'].values
train_labels = train_ground_truth['count'].values

# val
val_ground_truth = pd.read_csv(counting_data_dir + 'val/val_ground_truth.txt')
val_image_names = val_ground_truth['Image'].values
val_labels = val_ground_truth['count'].values

# 构建完整路径(test没有配对文件)
train_image_paths = [os.path.join(image_folder_counting_trains, name) for name in train_image_names]
val_image_paths = [os.path.join(image_folder_counting_val, name) for name in val_image_names]
test_image_paths = [os.path.join(image_folder_counting_test, name) for name in os.listdir(image_folder_counting_test)]

# 创建数据集
train_dataset = tf.data.Dataset.from_tensor_slices((train_image_paths, train_labels))
val_dataset = tf.data.Dataset.from_tensor_slices((val_image_paths, val_labels))
test_dataset = tf.data.Dataset.from_tensor_slices(test_image_paths)

# 数据预处理
# 预处理函数
def preprocess(path, label=None):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)  # 将图像解码为张量, decode_jpeg
    image = tf.image.resize(image, [img_height, img_width])  # 重新设置尺寸
    image = image / 255.0  #归一化
    return image, label

train_dataset = train_dataset.map(preprocess)
val_dataset = val_dataset.map(preprocess)
test_dataset = test_dataset.map(preprocess)

# 设置每批64个
train_dataset.batch(batch_size)
val_dataset.batch(batch_size)
test_dataset.batch(batch_size)

# 打乱数据
train_dataset.shuffle(shuffle_size)
val_dataset.shuffle(shuffle_size)
test_dataset.shuffle(shuffle_size)

# 设置迭代次数
train_dataset.repeat(2)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<_RepeatDataset element_spec=(TensorSpec(shape=(180, 180, 3), dtype=tf.float32, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>